# Assignment 4

## Assumption
1. Rak of A is n

Implement the simplex algorithm to maximize the objective function, You need to implement the method discussed in class.

Input: CSV file with m+1 rows and n+1 column.
             The first row excluding the last element is the cost vector c of length n
             The last column excluding the top element is the constraint vector b of length m
             Rows two to m+1 and column one to n is the matrix A of size m*n

Output: You need to print the sequence of vertices visited and the value of the objective function at that vertex

## Used Functions


## imports
**Details of Libraries:**
   - **`numpy`** - Importing for the numerical computations, like matrix operations & vector manipulations.
   - **`csv`**- For reading & writing CSV files containing input data
   - **`warnings`** - For controlling & manage warnings during program execution

In [1]:
import csv
import numpy as np
from scipy.linalg import null_space
import warnings

## Reading CSV file
This `read_csv` function will read i/p CSV file & extracts required components for simplex algorithm. It will open file & convert data from that file into structured NumPy arrays-  first row going for parsing as initial feasible solution, second row as cost vector, &  an last column of remaining rows as right-hand side vector. Preceding columns from those rows form constraint matrix. This function makes sure that data is correctly structured & also ready for  the use in solving linear programming problem and also streamlining input process & minimizing manual errors.


In [2]:
# Suppress warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

# Function to read CSV input
def read_csv(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)

    # Extracting cost vector from the first row
    cost_vec = np.array([float(x) for x in data[0][:-1]])

    # Extracting RHS vector from the last column of remaining rows
    rhs_vec = np.array([float(row[-1]) for row in data[1:]])

    # Extracting constraints matrix from remaining rows
    A = np.array([[float(x) for x in row[:-1]] for row in data[1:]])

    return A, rhs_vec, cost_vec

## Printing Matrices

This function is for printing key components of an mathematical model which is including constraint matrix, initial feasible solution, cost vector & right hand side vector, in structure & readable format.


In [3]:
# Function to compute directions
def compute_directions(tight_rows):
    try:
        # Compute the negative inverse of the tight rows matrix
        return -np.linalg.inv(tight_rows)
    except np.linalg.LinAlgError:
        # Handle the case where the tight rows matrix is singular (non-invertible)
        print("Singular matrix encountered; cannot compute directions.")
        return None

##  Identifying Tight Rows in Constraint Matrix

`find_tight_rows` function identifies which constraints in matrix \( A \) are tight for given initial feasible solution. It perform this by computing the diff between result of dot product of \( A \) & initial solution, & right-hand side vector \( b \). If an absolute difference is smaller than the specified tolerance (\( \epsilon \)), then that row is considered tight. Function then separates matrix \( A \) into 2  part-: `tight_rows`, which will be containing rows where constraints are tight, & `untight_rows`, which contains rows where constraints are not tight. This distinction is vey important to determinine which constraints are active in that feasible region of solution.


In [4]:
# determining tight rows from constraint matrix
def find_tight_rows(A, feasible_sol, rhs_vec, epsilon=1e-8):
    # Checking if the diff is less than a small epsilon to identify tight rows
    tight_mask = np.abs(np.dot(A, feasible_sol) - rhs_vec) < epsilon

    # Selecting the rows where the constraint is tight
    tight_rows = A[tight_mask]

    # Selecting the rows where the constraint are not tight
    untight_rows = A[~tight_mask]

    return tight_mask, tight_rows, untight_rows

## Checking Degeneracy

This function is checking for degeneracy in the linear programming problem by comparing no. of tight constraints (that is rows) to rank of matrix which is formed by those constraints. The message is displayed, if degeneracy is detected.


In [5]:
# Function to check for degeneracy
def check_degeneracy(tight_rows):
    # Degeneracy occurs if the number of rows (constraints) exceeds the number of columns (variables)
    return tight_rows.shape[0] > tight_rows.shape[1]



This function is for handling degeneracy in linear programming. By finding the reduced direction for movement within the feasible region. It is detecting degeneracy by comparing the no.r of tight constraints with the rank of the system. If it is detected then it calculates direction using the null space of tight constraints matrix.


In [6]:
# Function to adjust RHS vector for non-degeneracy
def make_non_degenerate(rhs_vec, epsilon, factor=0.5):
    # Adjust epsilon and perturb RHS to break degeneracy
    epsilon *= factor
    adjusted_rhs = rhs_vec + np.array([epsilon ** (i + 1) for i in range(len(rhs_vec))])
    return adjusted_rhs, epsilon

## Transitioning to a Feasible Vertex

`feasible_to_vertex` function is designed for converting initial feasible solution into vertex of feasible region for linear programming problem. It will starts by calculating the cost at initial solution & then identifies tight constraints using our `find_tight_rows` function. If there are no tight constraints are present or rank of tight constraints is less than number of variables, function will determine direction for moving in solution space. This direction is derived either from null space of tight constraints or from the chosen randomly if no tight constraints exist. Function then will calculate step size (alpha usesd below) for moving in that direction which will be ensuring updated solution remains feasible. Process then repeats iteratively, updating solution & tracking visited vertices, until vertex is found or all the constraints are satisfied. This function is very crucial for generating feasible solutions that will guide simplex algorithm towards optimal solution.



Additionally, this function is including several debug statements that are helping for tracking progress of the algorithm. It is printing current solution, tight constraints, direction vectors, and the sequence of the vertices which are visited during this process. This is mostly helpful for oberving the issues or understanding an iterative steps of the algorithm.

The function is also handling degeneracy by invoking the same function. This will be ensuring that solution continues to progress. Even in cases where the system has redundant constraints or dependent constraints.

This iterative process is ensuring that algorithm converges towards a feasible vertex, while maintaining feasibility at each step. This is making it robust and efficient for use in the simplex method.



In [7]:
# Function to perform simplex algorithm from a feasible point to a vertex
def feasible_to_vertex(A, rhs_vec, init_sol, cost_vec, n):
    vertices, costs = [init_sol], [np.dot(cost_vec, init_sol)]
    current_sol = init_sol
    mask, tight_rows, untight_rows = find_tight_rows(A, init_sol, rhs_vec)
    rank = np.linalg.matrix_rank(tight_rows) if len(tight_rows) > 0 else 0

    # Return directly if the initial point is already a vertex
    if rank == n:
        return current_sol, costs, vertices

    iteration = 0
    while rank != n:
        iteration += 1
        if iteration > 10000:  # Prevent infinite loops
            raise RuntimeError("Maximum iterations reached without finding a vertex.")

        null_space_matrix = null_space(tight_rows) if len(tight_rows) > 0 else None
        u = null_space_matrix[:, 0] if null_space_matrix is not None else np.random.rand(A.shape[1])

        # Compute step sizes for all untight constraints
        alphas = [
            (b - np.dot(a, current_sol)) / np.dot(a, u)
            for b, a in zip(rhs_vec[~mask], untight_rows)
            if np.dot(a, u) > 0  # Only consider positive denominators
        ]

        if not alphas:  # If no valid step size, the region is unbounded
            print("The problem is unbounded.")
            return None, costs, vertices

        alpha = min(alphas)
        current_sol = current_sol + alpha * u

        mask, tight_rows, untight_rows = find_tight_rows(A, current_sol, rhs_vec)
        rank = np.linalg.matrix_rank(tight_rows) if len(tight_rows) > 0 else 0

        costs.append(np.dot(cost_vec, current_sol))
        vertices.append(current_sol)

    if not check_degeneracy(tight_rows):
        return current_sol, costs, vertices
    else:
        return None, costs, vertices


## Vertex Optimization Using Simplex Method
 The `vertex_optimization` function implements core of Simplex method for optimizing solution iteratively by moving from one vertex to next. For storing visited vertices and corresponding objective function values, our function starts by initializing lists. It begins with given initial feasible solution & cost associated with it.


The function is moving iteratively from the feasible vertex towards optimal solution. It is identifying tight constraints, computing direction from the null space & calculates step sizes for updation of solution. The process continues til no further progression is possible. Then signaling that either the optimal solution is reached or no feasible direction exists. This method track the visited vertices & their associated costs as sol moving towards an optimality.


In each iteration, the function -
1. Calls `find_tight_rows` function and identifies tight & untight constraints.
2. Computes the potential directions for movement purpose based on tight constraints using function `compute_directions`.
3. By checking which ones improve objective function , it Filters directions (positive dot product with cost vector).
4. If no improving direction is found then the algorithm terminates which is indicating optimal solution had reached.
5. If improving directions exist, then it is selecting first one & computes step sizes for untight constraints.
6. Smallest positive alpha is determining step size to move towards next vertex.
7. Solution is updated by moving along chosen direction & new vertex is added into the list of visited vertices.
8. The objective function value at new solution is computed & is stored. Loop continues till no improving direction is found which is indicating that optimal solution had reached.

Finally, function will return optimal solution, sequence of costs, and the vertices visited during optimization process.

This function forms the heart of Simplex algorithm hence most important, iterating through feasible solutions and then improving objective function at each step until the optimal vertex is found.


In [8]:
# Function to perform vertex-to-vertex transitions
def vertex_to_vertex(A, rhs_vec, init_sol, cost_vec):
    # Storing initial solution and respective cost
    vertices, costs = [init_sol], [np.dot(cost_vec, init_sol)]
    # Starting with the initial solution
    current_sol = init_sol

    while True:
        # finding which constraints are tight which is not
        mask, tight_rows, untight_rows = find_tight_rows(A, current_sol, rhs_vec)

        # Checking is the solution is degenerate
        if check_degeneracy(tight_rows):
            print("Degenerate solution encountered.")
            return None, costs, vertices

        # Selecting directions that low the cost
        directions = compute_directions(tight_rows).T
        positive_directions = [d for d in directions if np.dot(d, cost_vec) > 0]

        # if no direction then we are at optimal sol
        if not positive_directions:
            return current_sol, costs, vertices

         # calculating alpah step sizes
        u = positive_directions[0]
        alphas = [
            (b - np.dot(a, current_sol)) / np.dot(a, u)
            for b, a in zip(rhs_vec[~mask], untight_rows)
            if np.dot(a, u) > 0
        ]
        # if no a valid state then print problem is unbounded
        if not alphas:
            print("The problem is unbounded.")
            return None, costs, vertices

        alpha = min(alphas)
        current_sol = current_sol + alpha * u

        # storing
        costs.append(np.dot(cost_vec, current_sol))
        vertices.append(current_sol)

## Reading CSV File

This code is for reading data from csv file of the testcase. This file is containing linear programming problem's matrices. It extracts the constraint matrix, initial feasible solution, right hand side vector & cost vector finally prints them for inspection.


In [9]:
# Reading data from the CSV file
A, rhs_vec, cost_vec = read_csv('testcase_7.csv')
m, n = A.shape
epsilon = 0.1
attempts = 0

## Converting Initial Feasible Solution to a Vertex and Tracking Costs
This code is defining no. of variables \( n \) based on number of columns in that constraint matrix \(A\). It calls `feasible_to_vertex` function to convet initial feasible solution into the vertex of feasible region. function returns updated solution (`init_feasible_sol`), sequence of costs at each vertex (`costs`), and list of vertices visited during process (`vertices`). This step is essential for moving from an initial feasible solution to an optimal vertex while tracking objective function values throughout iterations.


In [10]:
while True:
    if attempts > 0:
        rhs_vec, epsilon = make_non_degenerate(rhs_vec, epsilon)

    # Finding an initial feasible vertex
    feasible_result = feasible_to_vertex(A, rhs_vec, np.zeros(n), cost_vec, n)
    if feasible_result[0] is None:
        attempts += 1
        break
    # performing vertex to vertex optimization
    init_vertex, feas_costs, feas_vertices = feasible_result
    vertex_result = vertex_to_vertex(A, rhs_vec, init_vertex, cost_vec)

    if vertex_result[0] is None:
        attempts += 1
        break
    # get optimal vertex and cost
    opt_vertex, opt_costs, opt_vertices = vertex_result
    break

# Print visited vertices and costs
print("Visited Vertices:")
for i, (v, c) in enumerate(zip(feas_vertices, feas_costs)):
    print(f"Iteration {i}: Cost = {c}, Vertex = {v}")

# Print optimal solution
print("\nOptimal vertex is as follows:")
print(f"Point: {opt_vertices[-1]}")
print(f"Value of objective function: {opt_costs[-1]}")

Visited Vertices:
Iteration 0: Cost = 0.0, Vertex = [0. 0.]
Iteration 1: Cost = -2.282617519858235, Vertex = [-0.28261752 -0.71738248]
Iteration 2: Cost = -3.0, Vertex = [-1.  0.]

Optimal vertex is as follows:
Point: [-1.5  0.5]
Value of objective function: -3.5
